## Lets start from eugenesiow/bart-paraphrase model

In [10]:
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

In [11]:
# selecting model checkpoint
model_checkpoint = "eugenesiow/bart-paraphrase"

In [12]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the BLUE metric
metric = load_metric("sacrebleu")

### Dataset

In [16]:
train_df = pd.read_csv('../data/internal/train.csv')
test_df = pd.read_csv('../data/internal/test.csv')
val_df = pd.read_csv('../data/internal/validation.csv')

In [10]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [17]:
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/332 [00:00<?, ?B/s]

In [22]:
prefix = "detoxify:"
source = "reference"
target = "translation"
max_input_length = 128
max_target_length = 128
batch_size = 32

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source]]
    targets = [ex for ex in examples[target]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
preprocess_function(train_df[:2])

{'input_ids': [[0, 17701, 4325, 4591, 35, 3684, 37, 18, 300, 16, 14, 33676, 219, 7040, 9, 39, 2], [0, 17701, 4325, 4591, 36856, 10836, 75, 47, 192, 6, 6507, 16, 5373, 2901, 37, 16670, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 700, 18, 300, 1085, 53, 39, 23523, 7040, 4, 2], [0, 113, 6968, 64, 75, 192, 6, 1308, 119, 18, 5373, 2901, 91, 24509, 4, 2]]}

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source}-to-{target}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/site-packages/transformers/utils/import_utils.py", line 872, in _get_module
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py", line 21, in <module>
    from .auto_factory import _BaseAutoBackbon

In [28]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/site-packages/transformers/utils/import_utils.py", line 872, in _get_module
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/damirnurtdinov/miniforge3/envs/env_pytorch/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py", line 21, in <module>
    from .auto_factory import _BaseAutoBackbon